In [1]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://olist'
)
load.load()

Loader - Benchmarker : Success download the benchmark dataset from https://petsard-benchmark.s3.amazonaws.com/olist.csv.


In [3]:
synth = Synthesizer(method='sdv-gaussiancopula')

In [5]:
synth.create(load.data)

Synthesizer (SDV - SingleTable): Metafile loading time: 0.1358 sec.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/sdv/single_table/base.py:79: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [6]:
synth.fit_sample()

Synthesizer (SDV - SingleTable): Fitting GaussianCopula.


InvalidDataError: The provided data does not match the metadata:
Key column 'customer_zip_code_prefix' contains repeating values: [1004, 1005, 1006, '+ 12283 more']

In [8]:
synth.Synthesizer.metadata

{
    "columns": {
        "customer_id": {
            "sdtype": "unknown",
            "pii": true
        },
        "customer_unique_id": {
            "sdtype": "unknown",
            "pii": true
        },
        "customer_zip_code_prefix": {
            "sdtype": "postcode",
            "pii": true
        },
        "customer_city": {
            "sdtype": "city",
            "pii": true
        },
        "customer_state": {
            "sdtype": "administrative_unit",
            "pii": true
        },
        "order_id": {
            "sdtype": "unknown",
            "pii": true
        },
        "order_status": {
            "sdtype": "categorical"
        },
        "order_purchase_timestamp": {
            "sdtype": "datetime"
        },
        "order_approved_at": {
            "sdtype": "datetime"
        },
        "order_delivered_carrier_date": {
            "sdtype": "datetime"
        },
        "order_delivered_customer_date": {
            "sdtype": "datetime"

In [10]:
load.data['customer_zip_code_prefix'].value_counts()

customer_zip_code_prefix
24220    158
22790    155
22793    154
24230    138
22775    127
        ... 
9334       1
74953      1
62795      1
13234      1
99043      1
Name: count, Length: 14976, dtype: int64

In [11]:
load.data.isna().sum(axis=0)

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1245
order_delivered_customer_date    2567
order_estimated_delivery_date       0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
dtype: int64

In [13]:
a = load.metadata.to_sdv()

In [17]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

In [19]:
stm = SingleTableMetadata()

In [20]:
stm.detect_from_dataframe(load.data)

In [22]:
gcs = GaussianCopulaSynthesizer(stm)

/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/sdv/single_table/base.py:79: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [23]:
gcs.fit(load.data)

InvalidDataError: The provided data does not match the metadata:
Key column 'customer_zip_code_prefix' contains repeating values: [1004, 1005, 1006, '+ 12283 more']

In [25]:
gcs = GaussianCopulaSynthesizer(stm.load_from_dict(load.metadata.to_sdv()))

In [26]:
gcs.fit(load.data)

In [28]:
gcs.sample(100)

/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:700: RuntimeWarning: invalid value encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,fe7c7875de096a5aa735e7ae90ec580c,605c9f2be41b1632bec3dd720bc4fdce,29454,rio de janeiro,SP,c767c6e164464c133c309cd34b800997,invoiced,1970-01-01 00:00:01,1970-01-01 00:00:01,NaT,...,4,credit_card,1,97.237106,2,9b526d1b3e58c9453221353c4368e075,6860153b69cc696d5dcfe1cdaaafcf62,1970-01-01 00:00:01,134.410507,6.318676
1,8de735c5b8fcb11696c84002643f902a,c86ab5ff2bac777c4e770a383feca4eb,1030,rio de janeiro,SP,bccbe20275c5ca847160e9ca42aa220d,shipped,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,2,credit_card,2,2.760342,2,c326c16c4e67b3648c8992954d628795,53243585a1d6dc2643021fd1853d8905,1970-01-01 00:00:01,4.295846,6.269789
2,7f3c2e6516da8ec689dd5729e8772c95,638f94a5b338750865b41d709135d045,28638,sao paulo,SP,57e0ef9b41be8a4b90a17f09290d6d47,processing,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,2,credit_card,1,41.661564,3,57aae7271c7c24614b0d663c6df83f35,a1043bafd471dff536d0c462352beb48,1970-01-01 00:00:01,53.869347,8.972650
3,97ccfffe894ddb0e47cedf509786189f,ba47fc1e9bf003964a40acdfdc46a7db,14917,niteroi,RJ,3f385ecc36597ee0221d131e42725177,delivered,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,2,credit_card,2,132.823242,2,cdd68c0ef3e507db79631336cf9ec285,ea8482cd71df3c1969d7b9473ff13abc,1970-01-01 00:00:01,67.748528,31.466948
4,8fda58f1e825efd45641ed85581f7d8b,eca8479257eba8c8caddc0617d9d587e,11529,campinas,MG,b56bcbb57a9be94114e2dfb18399193e,delivered,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,3,credit_card,1,234.003967,21,2fffbf105cbbc6e86144f00ecf571635,53e4c6e0f4312d4d2107a8c9cddf45cd,1970-01-01 00:00:01,197.308701,17.352381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,d5c98e2099bddd4681eef690abc896c2,f11915ba29bfeff33b0f5610b9631aeb,64762,betim,RR,2c8368619bc15c47d6fd7de27cdef0c6,delivered,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,5,credit_card,2,98.038895,2,1b8ee158f59c098470fad33f39660964,6f892e20a171e98efe17fdb971ff319b,1970-01-01 00:00:01,204.465302,10.667106
96,ebdd88ff5419875a1f95e53380c30ef6,f8d067e9d4482d1a32805f05320800be,94221,vitoria,DF,1b0bc086985717610dbca11d3324771c,canceled,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,3,credit_card,1,93.165085,8,7b85e3deef35afd6ebed5461ee8f0641,cca3071e3e9bb7d12640c9fbe2301306,1970-01-01 00:00:01,23.806675,22.718287
97,57d5846f41d30acf13bb29a957846838,f4ecf73dc97b08307cd85c109e491fa1,7407,sao paulo,SP,4836de5d09ae58d55773d2e0179dccba,canceled,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,2,credit_card,4,141.426193,2,9ce74f28b3b6dee34ad83b1ede956664,dbc22125167c298ef99da25668e1011f,1970-01-01 00:00:01,66.213860,40.544704
98,449da50336d9b1e5f45284b5e4647f48,29b36e8709db24db3e302e60a2245a1c,49832,anapolis,SP,a6b52290e2cab59e8e7c8382eef9e523,shipped,1970-01-01 00:00:01,1970-01-01 00:00:01,1970-01-01 00:00:01,...,7,credit_card,1,60.277103,2,5f622da714951c5ae86acf68c0a03d30,b44dbfbf62e52125f3c15d230569ea91,1970-01-01 00:00:01,81.219559,14.646670
